# Chapter 10: Cluster Randomization and Hierarchical Modeling

# Libraries and data

### Libraries

In [1]:
# Common libraries
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
import seaborn as sns

# Chapter-specific libraries
# To rescale numeric variables
from sklearn.preprocessing import MinMaxScaler
# To one-hot encode cat. variables
from sklearn.preprocessing import OneHotEncoder

### Data

In [2]:
hist_data_df = pd.read_csv('chap10-historical_data.csv')
exp_data_df = pd.read_csv('chap10-experimental_data.csv')

#Shifting center_ID to distinguish it from indices
hist_data_df['center_ID'] = hist_data_df['center_ID'] + 100 
exp_data_df['center_ID'] = exp_data_df['center_ID'] + 100 

#Removing the variable M6Spend we won't use in this chapter
del(hist_data_df['M6Spend'])
del(exp_data_df['M6Spend'])

# Introduction to hierarchical modeling

In [3]:
# Hierarchical analysis of historical data with center ID only as clustering variable
mixed = smf.mixedlm("call_CSAT ~ reason + age", data = hist_data_df, 
                   groups = hist_data_df["center_ID"])
print(mixed.fit().summary())

            Mixed Linear Model Regression Results
Model:              MixedLM Dependent Variable: call_CSAT    
No. Observations:   695205  Method:             REML         
No. Groups:         10      Scale:              1.1217       
Min. group size:    54203   Log-Likelihood:     -1026427.7247
Max. group size:    79250   Converged:          Yes          
Mean group size:    69520.5                                  
-------------------------------------------------------------
                   Coef. Std.Err.    z    P>|z| [0.025 0.975]
-------------------------------------------------------------
Intercept          3.899    0.335  11.641 0.000  3.243  4.556
reason[T.property] 0.199    0.003  74.786 0.000  0.194  0.205
age                0.020    0.000 176.747 0.000  0.020  0.020
Group Var          1.122    0.407                            



In [4]:
# Hierarchical analysis of historical data with center ID and rep ID as clustering variable
vcf = {"rep_ID": "0+C(rep_ID)"}
mixed2 = smf.mixedlm("call_CSAT ~ reason + age", 
                    data = hist_data_df, 
                    groups = hist_data_df["center_ID"],
                    re_formula='1',
                    vc_formula=vcf)
print(mixed2.fit().summary())

            Mixed Linear Model Regression Results
Model:             MixedLM  Dependent Variable:  call_CSAT   
No. Observations:  695205   Method:              REML        
No. Groups:        10       Scale:               0.3904      
Min. group size:   54203    Log-Likelihood:      -660424.9323
Max. group size:   79250    Converged:           Yes         
Mean group size:   69520.5                                   
-------------------------------------------------------------
                   Coef. Std.Err.    z    P>|z| [0.025 0.975]
-------------------------------------------------------------
Intercept          3.901    0.367  10.639 0.000  3.182  4.620
reason[T.property] 0.200    0.002 126.789 0.000  0.196  0.203
age                0.020    0.000 298.302 0.000  0.020  0.020
Group Var          1.304    0.976                            
rep_ID Var         0.776    0.131                            



# Determining random assignment and sample size/power

In [5]:
# Functions for simulation

def hlm_metric_fun(dat_df):
    vcf = {"rep_ID": "0+C(rep_ID)"}
    h_mod = smf.mixedlm("call_CSAT ~ reason + age + group", 
                    data = dat_df, 
                    groups = dat_df["center_ID"],
                    re_formula='1',
                    vc_formula=vcf)
    coeff = h_mod.fit().fe_params.values[2]
    return coeff

def boot_CI_fun(dat_df, metric_fun = hlm_metric_fun, B = 20, conf_level = 9/10):
    Ncalls_rep = 1200
    coeff_boot = []
    # Calculate coeff of interest for each simulation
    for b in range(B):
        boot_df = dat_df.groupby("rep_ID").sample(n=Ncalls_rep,
                                                  replace=True).\
        reset_index(drop= True)
        coeff = metric_fun(boot_df)
        coeff_boot.append(coeff)
    #Extract confidence interval
    coeff_boot.sort()
    offset = round(B * (1 - conf_level) / 2)
    confint = [coeff_boot[offset], coeff_boot[-(offset+1)]]
    return confint

def decision_fun(dat_df, metric_fun, B = 20, conf_level = 0.9):
    boot_CI = boot_CI_fun(dat_df, metric_fun, B = B, conf_level = conf_level)
    decision = 1 if boot_CI[0] > 0  else 0
    return decision

## Random assignment

In [8]:
# Aggregating data to center level
center_data_df = hist_data_df.groupby('center_ID').agg(
        nreps = ('rep_ID', lambda x: x.nunique()),
        avg_call_CSAT = ("call_CSAT", "mean"),
        avg_age=("age", "mean"),
        pct_reason_pmt=('reason', 
                        lambda x: sum(1 if r=='payment' else 0 for r in x)/len(x))
        )

#Reformatting variables as needed
center_data_df['nreps'] = center_data_df.nreps.astype(float)

center_data_df.head()

,nreps,avg_call_CSAT,avg_age,pct_reason_pmt
center_ID,,,,
101,18.0,3.664430,39.962880,0.601027
102,21.0,3.958169,39.959532,0.599237
103,22.0,4.030376,39.981830,0.599508
104,15.0,5.296561,40.063354,0.599690
105,21.0,5.921405,39.977681,0.600679


In [10]:
### Function to prep the data
def strat_prep_fun(dat_df):

    #Extracting components of the data
    num_df = dat_df.copy().loc[:,dat_df.dtypes=='float64'] #Numeric vars
    center_ID = [i for i in dat_df.index]

    #Normalizing all numeric variables to [0,1]
    scaler = MinMaxScaler()
    scaler.fit(num_df)
    num_np = scaler.transform(num_df)
    
    return center_ID, num_np
    
def pair_fun(dat_df, K = 2):
    
    match_len = K - 1 # Number of matches we want to find
    match_idx = match_len - 1 # Accounting for 0-indexing
    
    center_ID, data_np = strat_prep_fun(dat_df)
    N = len(data_np)
    
    #Calculate distance matrix
    from scipy.spatial import distance_matrix
    d_mat = distance_matrix(data_np, data_np)
    np.fill_diagonal(d_mat,N+1)
    # Set up variables
    available = [i for i in range(N)]
    available_temp = available.copy()
    matches_lst = []
    lim = int(N/match_len)
    
    closest = np.argpartition(d_mat, kth=match_idx,axis=1)
    
    for n in available:
        if len(matches_lst) == lim: break
        if n in available_temp:
            for match_lim in range(match_idx,N-1):
                possible_matches = closest[n,:match_lim].tolist()
                matches = list(set(available_temp) & set(possible_matches))
                if len(matches) == match_len:
                    matches.append(n)
                    matches_lst.append(matches)
                    available_temp \
                    = [m for m in available_temp if m not in matches]
                    break
                else:
                    closest[n,:] = np.argpartition(d_mat[n,:], kth=match_lim)
    #Map center indices to their proper IDs
    matches_id_lst = [[center_ID[k[0]],center_ID[k[1]]] for k in matches_lst]
    return np.array(matches_id_lst)
pair_fun(center_data_df)

array([[102, 101],
       [106, 103],
       [107, 104],
       [110, 105],
       [109, 108]])

## Power analysis

In [10]:
##### Simulation function #####
def power_sim_fun(dat_df, metric_fun = hlm_metric_fun, Ncalls_rep = 1000, eff_size = 1, B = 20, conf_level = 0.9):
    
    #Extract the stratified pairs
    stratified_pairs = stratified_assgnt_fun(dat_df, K=2)
    Npairs = len(stratified_pairs)
    Nperm = 2 ** Npairs
    power_list = []
    
    for m in dat_df.month.unique():
        #Sample down the data
        sample_data_df = dat_df.loc[dat_df.month==m,]
        sample_data_df = sample_data_df.groupby('rep_ID').sample(n=Ncalls_rep)\
            .reset_index(drop = True)
        for perm in range(Nperm):
            bin_str = f'{perm:0{Npairs}b}'
            idx = np.array([[i for i in range(Npairs)],
                            [int(d) for d in bin_str]]).T
            treat = [stratified_pairs[tuple(idx[i])] for i in range(Npairs)]
            
            sim_data_df = sample_data_df.copy()
            sim_data_df['group'] = 'ctrl'
            sim_data_df.loc[(sim_data_df.center_ID.isin(treat)),'group']\
                = 'treat'
            
            sim_data_df.loc[(sim_data_df.group=='treat'),'call_CSAT'] =\
                sim_data_df.loc[(sim_data_df.group=='treat'),'call_CSAT'] + eff_size
                
            sim_data_df.loc[(sim_data_df.call_CSAT > 10), 'call_CSAT'] = 10
            
            # Option 1: extract CIs for visualization
            #sim_CI = boot_CI_fun(sim_data_df, lm_metric_fun)
            #power_list.append(sim_CI)
            
            # Option 2: calculate decision for overall power determination
            D = decision_fun(sim_data_df, metric_fun, B = B, conf_level = conf_level)
            power_list.append(D)
            
    return power_list

# Analyzing the experiment

In [11]:
##### Analysis of experimental data #####

coeff = hlm_metric_fun(exp_data_df)
print(coeff)

hlm_CI = boot_CI_fun(exp_data_df, hlm_metric_fun)
print(hlm_CI)

0.477903237163797
[0.4743435990678476, 0.48186949193475165]
